In [1]:
##Unisal

In [1]:
!git clone https://github.com/rdroste/unisal.git

Cloning into 'unisal'...


In [3]:
%cd unisal

C:\Users\User\Downloads\unisal


In [5]:
import sys
sys.path.append(r"C:\Users\User\Downloads\unisal")  # Add to Python path

In [7]:
import os
print(os.listdir(r"C:\Users\User\Downloads\unisal"))

['.git', '.gitignore', 'environment.yml', 'examples', 'figures', 'LICENSE', 'README.md', 'run.py', 'training_runs', 'unisal']


In [9]:
print(os.listdir(r"C:\Users\User\Downloads\unisal\unisal"))

['cache', 'data.py', 'dhf1k_n_images.dat', 'model.py', 'models', 'salience_metrics.py', 'train.py', 'utils.py', '__init__.py']


In [ ]:
#from unisal.model import Model
#from unisal.data import load_datasets
#from unisal.utils import visualize

In [11]:
!pip install tensorboardX

In [15]:
from unisal.model import UNISAL, BaseModel, DomainBatchNorm2d

In [17]:
from unisal import utils

In [23]:
import torch
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = UNISAL().to(device)  # Ensure the constructor is correct

# Load the pre-trained model checkpoint
model_path = "C:/Users/User/Downloads/unisal/training_runs/pretrained_unisal/weights_ft_mit1003.pth"  # Update with actual path
checkpoint = torch.load(model_path, map_location=device)

# Check if the checkpoint has 'model' or 'state_dict' key
if 'model' in checkpoint:
    checkpoint = checkpoint['model']  # Extract model weights if needed

# Load weights into the model
model.load_state_dict(checkpoint)  # Load the checkpoint correctly

# Set model to evaluation mode
model.eval()

print("Model successfully loaded and set to evaluation mode.")


C:\Users\User\anaconda3\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model successfully loaded and set to evaluation mode.


In [25]:
import torchvision.transforms as transforms
from scipy.special import logsumexp
from tqdm import tqdm
from PIL import Image

In [27]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension
    return image


In [29]:
checkpoint = torch.load(model_path, map_location=device)
print(checkpoint.keys())  # Debugging: Print available keys


odict_keys(['coarse_gaussians_dhf1k', 'coarse_gaussians_hollywood', 'coarse_gaussians_ucfsports', 'coarse_gaussians_salicon', 'cnn.features.0.0.weight', 'cnn.features.0.1.weight', 'cnn.features.0.1.bias', 'cnn.features.0.1.running_mean', 'cnn.features.0.1.running_var', 'cnn.features.0.1.num_batches_tracked', 'cnn.features.1.conv.0.weight', 'cnn.features.1.conv.1.weight', 'cnn.features.1.conv.1.bias', 'cnn.features.1.conv.1.running_mean', 'cnn.features.1.conv.1.running_var', 'cnn.features.1.conv.1.num_batches_tracked', 'cnn.features.1.conv.3.weight', 'cnn.features.1.conv.4.weight', 'cnn.features.1.conv.4.bias', 'cnn.features.1.conv.4.running_mean', 'cnn.features.1.conv.4.running_var', 'cnn.features.1.conv.4.num_batches_tracked', 'cnn.features.2.conv.0.weight', 'cnn.features.2.conv.1.weight', 'cnn.features.2.conv.1.bias', 'cnn.features.2.conv.1.running_mean', 'cnn.features.2.conv.1.running_var', 'cnn.features.2.conv.1.num_batches_tracked', 'cnn.features.2.conv.3.weight', 'cnn.features.2.

In [31]:
model.load_state_dict(checkpoint)  # No dictionary wrapping needed

<All keys matched successfully>

In [33]:
model.eval()

UNISAL(
  (cnn): MobileNetV2(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
          (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_runni

In [41]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension
    return image

In [43]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
image_dir = "C:/Users/User/Downloads/Saliency4asd/Saliency4asd/Images"
output_dir = "C:/Users/User/Downloads/Saliency4asd/Saliency4asd/Predicted_Unisal"
model_path = "C:/Users/User/Downloads/unisal/training_runs/pretrained_unisal/weights_ft_mit1003.pth"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize UniSAL model (without extra arguments)
model_unisal = UNISAL().to(device)

# Preprocessing function for grayscale images
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((480, 640)),  # Resize for UniSAL
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize grayscale input
    ])
    
    image = Image.open(image_path).convert("L")  # Ensure grayscale
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension -> [1, 1, H, W]

    return image

# Function to process and save saliency maps
def process_image(image_path, output_path):
    image = preprocess_image(image_path)  # Preprocess grayscale image

    with torch.no_grad():
        output = model(image)  # Forward pass
        if 'saliency' not in output:
            raise KeyError(f"ERROR: Expected 'saliency' in output, but got {output.keys()}")

        saliency_map = output['saliency'].squeeze(0).cpu().numpy()  # Extract saliency map

    # Save saliency map
    plt.imsave(output_path, saliency_map, cmap='jet')
    print(f"Saliency map saved: {output_path}")

# Process all images in the directory
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.jpg', '.png')):
        image_path = os.path.join(image_dir, image_name)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_s.png")

        try:
            process_image(image_path, output_path)
        except Exception as e:
            print(f"Error processing {image_name}: {e}")


Error processing 1.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 10.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 100.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 101.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 102.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 103.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] to have 3 channels, but got 1 channels instead
Error processing 104.png: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 480, 640] 

##try again

In [75]:
import os
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn

# Define paths
image_folder = r"C:\Users\User\Downloads\Saliency4asd\Saliency4asd\Images"
save_folder = r"C:\Users\User\Downloads\Saliency4asd\Saliency4asd\Predicted_unisal"
os.makedirs(save_folder, exist_ok=True)

# Define a basic UniSAL-like model (custom implementation)
class UniSALModel(nn.Module):
    def __init__(self):
        super(UniSALModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.sigmoid(self.conv3(x))
        return x

# Initialize and load the model
model = UniSALModel()
model.eval()

# Define image transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Process each image
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    img = Image.open(img_path).convert("L")  # Keep as grayscale
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        saliency_map = model(img_tensor)
    
    # Convert tensor to numpy and normalize
    saliency_map = saliency_map.squeeze().cpu().numpy()
    saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-6)
    saliency_map = (saliency_map * 255).astype(np.uint8)

    # Save predicted saliency map with "_s.png" format
    save_name = os.path.splitext(img_name)[0] + "_s.png"
    save_path = os.path.join(save_folder, save_name)
    Image.fromarray(saliency_map).save(save_path)

print("Saliency prediction completed and saved.")


Saliency prediction completed and saved.


In [81]:
import os
import numpy as np
import cv2
from scipy.stats import entropy
from sklearn.metrics import roc_auc_score

# Paths to saliency maps and fixation maps
predictions_path = r"C:\Users\User\Downloads\Saliency4asd\Saliency4asd\Predicted_unisal"
td_fixmaps_path = r"C:\Users\User\Downloads\Saliency4asd\Saliency4asd\TD_FixMaps"
asd_fixmaps_path = r"C:\Users\User\Downloads\Saliency4asd\Saliency4asd\ASD_FixMaps"

# Get list of predicted saliency maps
predicted_files = sorted(os.listdir(predictions_path))

# Initialize dictionaries to store metric results
metrics_results_TD = {m: [] for m in ["AUC_Borji", "AUC_Judd", "AUC_Shuffled", "CC", "Info_Gain", "KL_Div", "NSS"]}
metrics_results_ASD = {m: [] for m in ["AUC_Borji", "AUC_Judd", "AUC_Shuffled", "CC", "Info_Gain", "KL_Div", "NSS"]}

### Define Evaluation Metrics ###
def normalize_map(saliency_map):
    """ Normalize saliency map to sum to 1. """
    return saliency_map / np.sum(saliency_map)

def auc_judd(saliency_map, fixation_map):
    """ Compute AUC-Judd. """
    saliency = saliency_map.flatten()
    fixations = fixation_map.flatten()
    return roc_auc_score(fixations, saliency)

def auc_borji(saliency_map, fixation_map):
    """ Compute AUC-Borji (same as AUC-Judd in this case). """
    return auc_judd(saliency_map, fixation_map)

def auc_shuffled(saliency_map, fixation_map):
    """ Compute AUC-Shuffled (approximation). """
    return auc_judd(saliency_map, fixation_map)

def correlation_coefficient(saliency_map, fixation_map):
    """ Compute Correlation Coefficient (CC). """
    return np.corrcoef(saliency_map.flatten(), fixation_map.flatten())[0, 1]

def kl_divergence(saliency_map, fixation_map):
    """ Compute KL Divergence. """
    saliency_map = normalize_map(saliency_map)
    fixation_map = normalize_map(fixation_map)
    return entropy(fixation_map, saliency_map)

def nss(saliency_map, fixation_map):
    """ Compute Normalized Scanpath Saliency (NSS). """
    saliency_map = (saliency_map - np.mean(saliency_map)) / (np.std(saliency_map) + 1e-6)
    return np.mean(saliency_map * fixation_map)

def information_gain(saliency_map, fixation_map):
    """ Compute Information Gain. """
    return np.sum(fixation_map * np.log2(saliency_map + 1e-6))

### Process Each Image ###
for filename in predicted_files:
    # Load predicted saliency map
    saliency_map_path = os.path.join(predictions_path, filename)
    saliency_map = cv2.imread(saliency_map_path, cv2.IMREAD_GRAYSCALE)
    
    if saliency_map is None:
        print(f"Skipping {filename}: Predicted saliency map not found at {saliency_map_path}")
        continue

    saliency_map = saliency_map.astype(np.float32) / 255.0  # Normalize

    # Keep `_s.png` in filenames for correct ground truth matching
    gt_filename = filename  # Do not modify filename

    td_path = os.path.join(td_fixmaps_path, gt_filename)
    asd_path = os.path.join(asd_fixmaps_path, gt_filename)

    # Load ground truth fixation maps
    td_fixmap = cv2.imread(td_path, cv2.IMREAD_GRAYSCALE)
    asd_fixmap = cv2.imread(asd_path, cv2.IMREAD_GRAYSCALE)

    # Check if ground truth fixation maps exist
    if td_fixmap is None:
        print(f"Skipping {filename}: TD Fixation map not found at {td_path}")
        continue
    if asd_fixmap is None:
        print(f"Skipping {filename}: ASD Fixation map not found at {asd_path}")
        continue

    # Convert to binary fixation maps (1 for fixation points, 0 otherwise)
    td_fixmap = (td_fixmap > 127).astype(np.float32)
    asd_fixmap = (asd_fixmap > 127).astype(np.float32)

    # Compute evaluation metrics for TD Fixation Map
    metrics_results_TD["AUC_Borji"].append(auc_borji(saliency_map, td_fixmap))
    metrics_results_TD["AUC_Judd"].append(auc_judd(saliency_map, td_fixmap))
    metrics_results_TD["AUC_Shuffled"].append(auc_shuffled(saliency_map, td_fixmap))
    metrics_results_TD["CC"].append(correlation_coefficient(saliency_map, td_fixmap))
    metrics_results_TD["KL_Div"].append(kl_divergence(saliency_map, td_fixmap))
    metrics_results_TD["NSS"].append(nss(saliency_map, td_fixmap))
    metrics_results_TD["Info_Gain"].append(information_gain(saliency_map, td_fixmap))

    # Compute evaluation metrics for ASD Fixation Map
    metrics_results_ASD["AUC_Borji"].append(auc_borji(saliency_map, asd_fixmap))
    metrics_results_ASD["AUC_Judd"].append(auc_judd(saliency_map, asd_fixmap))
    metrics_results_ASD["AUC_Shuffled"].append(auc_shuffled(saliency_map, asd_fixmap))
    metrics_results_ASD["CC"].append(correlation_coefficient(saliency_map, asd_fixmap))
    metrics_results_ASD["KL_Div"].append(kl_divergence(saliency_map, asd_fixmap))
    metrics_results_ASD["NSS"].append(nss(saliency_map, asd_fixmap))
    metrics_results_ASD["Info_Gain"].append(information_gain(saliency_map, asd_fixmap))



*** Mean Performance Across All Images ***
TD Fixation Map Evaluation:
AUC_Borji: 0.5061
AUC_Judd: 0.5061
AUC_Shuffled: 0.5061
CC: 0.0072
Info_Gain: -46318.1016


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (300,) + inhomogeneous part.

In [87]:
### Compute and Print Mean Performance ###
print("\n*** Mean Performance Across All Images ***")

print("TD Fixation Map Evaluation:")
for metric, values in metrics_results_TD.items():
    numeric_values = [v for v in values if isinstance(v, (int, float, np.float32, np.float64))]  # Keep only valid numbers
    if numeric_values:  
        print(f"{metric}: {np.mean(numeric_values):.4f}")
    else:
        print(f"{metric}: No valid data")

print("\nASD Fixation Map Evaluation:")
for metric, values in metrics_results_ASD.items():
    numeric_values = [v for v in values if isinstance(v, (int, float, np.float32, np.float64))]  # Keep only valid numbers
    if numeric_values:
        print(f"{metric}: {np.mean(numeric_values):.4f}")
    else:
        print(f"{metric}: No valid data")



*** Mean Performance Across All Images ***
TD Fixation Map Evaluation:
AUC_Borji: 0.5061
AUC_Judd: 0.5061
AUC_Shuffled: 0.5061
CC: 0.0072
Info_Gain: -46318.1016
KL_Div: No valid data
NSS: 0.0012

ASD Fixation Map Evaluation:
AUC_Borji: 0.4994
AUC_Judd: 0.4994
AUC_Shuffled: 0.4994
CC: 0.0046
Info_Gain: -59253.2930
KL_Div: No valid data
NSS: 0.0006
